In [7]:
import os
from dotenv import load_dotenv

In [8]:
# Load environment variables from a .env file
load_dotenv()

video_url = 'https://www.youtube.com/watch?v=OPb-nj9-Qsc&t=1189s'
print(video_url)

https://www.youtube.com/watch?v=OPb-nj9-Qsc&t=1189s


In [14]:
from googleapiclient.discovery import build
import re

# Replace with your actual API key
googleKey = os.getenv('googleKey')

# Create a YouTube API client
youtube = build('youtube', 'v3', developerKey=googleKey)

def remove_time_tag(url):
    # Remove time tag if present
    return re.sub(r'[&?]t=\d+[hms]?(&?)', r'\1', url)

def get_video_info(url):
    # Remove time tag
    clean_url = remove_time_tag(url)
    
    # Extract video ID
    video_id_match = re.search(r'(?:v=|\/)([0-9A-Za-z_-]{11}).*', clean_url)
    if not video_id_match:
        return None, None
    
    video_id = video_id_match.group(1)
    
    # Get video details
    try:
        response = youtube.videos().list(
            part='snippet',
            id=video_id
        ).execute()
        
        if not response['items']:
            return None, None
        
        channel_id = response['items'][0]['snippet']['channelId']
        channel_title = response['items'][0]['snippet']['channelTitle']
        
        return channel_title, channel_id, video_id
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

# Example usage
#url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ&t=1m30s"
channel_name, channel_id, video_id = get_video_info(video_url)

if channel_name and channel_id:
    print(f"Video ID: {video_id}")
    print(f"Channel Name: {channel_name}")
    print(f"Channel ID: {channel_id}")
else:
    print("Could not retrieve channel information.")


Video ID: OPb-nj9-Qsc
Channel Name: 亂gad - randgad
Channel ID: UCFPAwIW39LR4JwwXFr8DhpQ


In [19]:
import requests

def get_video_duration(video_id, api_key):
    url = f"https://www.googleapis.com/youtube/v3/videos?id={video_id}&part=contentDetails&key={api_key}"
    response = requests.get(url)
    data = response.json()
    
    if 'items' in data and len(data['items']) > 0:
        duration = data['items'][0]['contentDetails']['duration']  # e.g., PT4M54S
        
        # write the response.json() to a json file
        import json
        with open('response.json', 'w') as f:
            json.dump(data, f) # write the response.json() to a json file
            print(f"response.json() has been written to response.json file")
            f.close()
            print(f"response.json() file has been closed")
        
        return duration
    else:
        return None

# Example usage
api_key = os.getenv('googleKey')
#video_id = 'VIDEO_ID_HERE'
duration = get_video_duration(video_id, api_key)
print(f"Duration: {duration}")
    



response.json() has been written to response.json file
response.json() file has been closed
Duration: PT1H14M30S


In [49]:
from googleapiclient.discovery import build


def get_channel_videos(channel_id):
    videos = []
    next_page_token = None
    
    while True:
        # Request channel uploads playlist
        res = youtube.channels().list(id=channel_id, 
                                      part='contentDetails').execute()
        playlist_id = res['items'][0]['contentDetails']['relatedPlaylists']['uploads']
        
        # Request playlist items
        res = youtube.playlistItems().list(playlistId=playlist_id,
                                           part='snippet',
                                           maxResults=50,
                                           pageToken=next_page_token).execute()
        
        # Process video items
        for item in res['items']:
            video_id = item['snippet']['resourceId']['videoId']
            title = item['snippet']['title']
            publish = item['snippet']['publishedAt']
            #publishDate = publishAt.split('T')[0]
            
            # Request video details to get duration
            video_details = youtube.videos().list(id=video_id, part='contentDetails').execute()
            duration = video_details['items'][0]['contentDetails']['duration']
            
            videos.append({'video_id': video_id, 'publish': publish, 'duration': duration, 'title': title})
        
        # Check for next page
        next_page_token = res.get('nextPageToken')
        if not next_page_token:
            break
    
    return videos


video_list = get_channel_videos(channel_id)




In [50]:
import psycopg2
import isodate

# Assuming video_list and channel_name are defined elsewhere in your code
# Print the list of the last 3 videos
for video in video_list[:3]:
    print(f"VideoID: {video['video_id']}, Channel: {channel_name}, Published: {video['publish']}, Duration: {video['duration']}, Title: {video['title']}")

# Connect to your postgres DB
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="wHy252",
    host="localhost",
    port="5432"
)

# Open a cursor to perform database operations
cur = conn.cursor()

# Drop table if exists and create table if not exists
cur.execute("""
DROP TABLE IF EXISTS videos;
CREATE TABLE IF NOT EXISTS videos (
    video_id VARCHAR PRIMARY KEY,
    channel_name VARCHAR,
    publish_at TIMESTAMP,
    duration INTERVAL,
    title TEXT
)
""")

# Insert data into the table
for video in video_list:
    duration = isodate.parse_duration(video['duration'])
    title = video['title'].replace('\u200c', '')  # Remove \u200c from title
    cur.execute("""
    INSERT INTO videos (video_id, channel_name, publish_at, duration, title)
    VALUES (%s, %s, %s, %s, %s)
    ON CONFLICT (video_id) DO NOTHING
    """, (video['video_id'], channel_name, video['publish'], duration, title))

# Commit the transaction
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()


VideoID: OPb-nj9-Qsc, Channel: 亂gad - randgad, Published: 2025-01-06T05:13:00Z, Duration: PT1H14M30S, Title: 亂‌‌‌gad‌‌‌ ‌‌‌‌‌第745集 - 阿Ed 多倫多之旅 / Switch 2 / Starship IFT 7 / Deepseek v3 / AI 影片 Sora 終於有得試
VideoID: PTBMjuWgSRA, Channel: 亂gad - randgad, Published: 2025-01-04T17:02:18Z, Duration: PT1H18M35S, Title: 亂‌‌‌gad‌‌‌ ‌‌‌‌‌第745集 - 阿Ed 多倫多之旅 / Switch 2 / Starship IFT 7 / Deepseek v3 / AI 影片 Sora 終於有得試
VideoID: iUm0rBbHfSI, Channel: 亂gad - randgad, Published: 2024-12-28T17:32:12Z, Duration: PT53M1S, Title: 亂‌‌‌gad‌‌‌ ‌‌‌‌‌第744集 - Excel 電競 MEWC 2024 / OpenAI 轉型做盈利公司 / $19 藍牙耳機有冇伏


In [53]:
# Connect to your postgres DB
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="wHy252",
    host="localhost",
    port="5432"
)

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a query to fetch data from the table
cur.execute("SELECT video_id, channel_name, publish_at, ROUND(EXTRACT(EPOCH FROM duration)/60) AS duration_minutes, title FROM videos LIMIT 5")

# Fetch and print the results
rows = cur.fetchall()
for row in rows:
    print(row)

# Close the cursor and connection
cur.close()
conn.close()

('OPb-nj9-Qsc', '亂gad - randgad', datetime.datetime(2025, 1, 6, 5, 13), Decimal('75'), '亂gad 第745集 - 阿Ed 多倫多之旅 / Switch 2 / Starship IFT 7 / Deepseek v3 / AI 影片 Sora 終於有得試')
('PTBMjuWgSRA', '亂gad - randgad', datetime.datetime(2025, 1, 4, 17, 2, 18), Decimal('79'), '亂gad 第745集 - 阿Ed 多倫多之旅 / Switch 2 / Starship IFT 7 / Deepseek v3 / AI 影片 Sora 終於有得試')
('iUm0rBbHfSI', '亂gad - randgad', datetime.datetime(2024, 12, 28, 17, 32, 12), Decimal('53'), '亂gad 第744集 - Excel 電競 MEWC 2024 / OpenAI 轉型做盈利公司 / $19 藍牙耳機有冇伏')
('vPs13eLXsI0', '亂gad - randgad', datetime.datetime(2024, 12, 28, 8, 40), Decimal('53'), '亂gad 第744集 ~ Excel 電競 MEWC 2024 / OpenAI 轉型做盈利公司 / HK$19 藍牙耳機有冇伏')
('6ptd62EOPg8', '亂gad - randgad', datetime.datetime(2024, 12, 21, 17, 43), Decimal('48'), '亂gad 第743集 - Apple Broadcom AI 晶片 / TP-Link 可能被美國禁用 / Lenovo Legion Go S / OpenAI 打電話玩 AI')
